In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

from PyPDF2 import PdfReader

In [ ]:
# Load model directly
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("openbmb/MiniCPM4.1-8B", trust_remote_code=True, torch_dtype="auto")

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="openbmb/MiniCPM4.1-8B", trust_remote_code=True)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openbmb/MiniCPM4.1-8B")

In [ ]:
import torch
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    device=0 if torch.cuda.is_available() else -1
)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
pdf_path="/content/w31161.pdf"
text=""
reader=PdfReader(pdf_path)
for page in reader.pages:
    text+=page.extract_text()

splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks=splitter.split_text(text)

vectorstore = FAISS.from_texts(chunks, embedding=embeddings)

In [ ]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
retriever=vectorstore.as_retriever()

template="""
You are a personalized AI tutor.
    Student profile: Beginner in Computer Science.
    Always explain in simple steps with examples.

    Context: {context}
    Question: {question}
    Helpful Answer:"""
prompt=PromptTemplate(template=template,input_variables=["context","question"])
memory=ConversationBufferMemory(memory_key="chat_history",return_messages=True)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt}
    )

In [ ]:
quiz_template = """
    You are a quiz generator AI tutor.
    Create 3 multiple-choice questions on the topic below.
    Topic: {topic}
    """
quiz_prompt = PromptTemplate(
    input_variables=["topic"],
    template=quiz_template
)
LLMChain(llm=llm, prompt=quiz_prompt)

In [ ]:
student_lvl="""
  you are an ai tutor.
  student level:{level}
  Learning Goal: {goal}
  Preferred Pace: {pace}"""

level_prompt=PromptTemplate(template=student_lvl,input_variables=["level","goal","pace"])
LLMChain(llm=llm,prompt=level_prompt)